# Target

This is the target of this notebook:
- Extract public data from government purchase
- Transform the data
- Load the data into a file

# Install and import the libraries

In [14]:
! pip install pyspark pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import requests
import io
import pandas as pd
from pyspark.sql import SparkSession

# Extract public data

In [16]:
url = 'http://compras.dados.gov.br/servicos/v1/servicos.csv?descricao=educação'

In [17]:
data = requests.get(url).content

In [18]:
type(data)

bytes

# Transforming the data

#### Pandas
- Non distributed data processing

In [19]:
parse_data = io.StringIO(data.decode("UTF-8"))

In [20]:
df_pandas = pd.read_csv(parse_data)

In [21]:
df_pandas.head()

,Código,Descrição,Unidade medida,CPC,Seção,Divisão,Grupo,Classe,Subclasse
0,14311,Orientação / Educação - Atividade Física ...,UNHOMEM/H MÊS,9659,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...","96: SERVIÇOS RECREACIONAIS, CULTURAIS E ESPORT...",965: SERVIÇOS RELACIONADOS COM ESPORTES E SERV...,9659: OUTROS ESPORTES E SERVIÇOS DE ESPORTES R...,NaN
1,15946,Serviço Penitenciário - Reeducação e Reabilita...,UN,979,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",97: OUTROS SERVIÇOS ...,979: OUTROS SERVIÇOS DIVERSOS/MISCELÂNEA ...,NaN,NaN
2,18481,Consultoria e Assessoria - Educação ...,UN,8319,"8: SERVIÇOS DE ARQUITETURA, DESENHOS TÉCNICOS,...","83: OUTROS SERVIÇOS DE NEGÓCIOS, TÉCNICOS E PR...",831: SERVIÇOS DE CONSULTORIA E DE GERÊNCIA/GES...,"8319: OUTROS SERVIÇOS DE GERÊNCIA/GESTÃO, EXCE...",NaN
3,19321,Curso / Treinamento Educação - Distância ...,UN,929,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",92: SERVIÇOS DA INSTRUÇÃO/ EDUCAÇÃO/ ENSINO ...,929: OUTROS SERVIÇOS DE EDUCAÇÃO E TREINAMENTO...,NaN,NaN


## Spark
- Distributed data processing

In [22]:
spark = SparkSession.builder.appName("government_purchase").getOrCreate()

In [23]:
df = spark.createDataFrame(df_pandas)

In [24]:
df.printSchema()

root
 |-- Código: long (nullable = true)
 |-- Descrição: string (nullable = true)
 |-- Unidade medida: string (nullable = true)
 |-- CPC: long (nullable = true)
 |-- Seção: string (nullable = true)
 |-- Divisão: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Classe: string (nullable = true)
 |-- Subclasse: double (nullable = true)



In [25]:
df.show(20)

+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
|Código|           Descrição|Unidade medida| CPC|               Seção|             Divisão|               Grupo|              Classe|Subclasse|
+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
| 14311|Orientação / Educ...| UNHOMEM/H MÊS|9659|9: SERVIÇOS DA CO...|96: SERVIÇOS RECR...|965: SERVIÇOS REL...|9659: OUTROS ESPO...|      NaN|
| 15946|Serviço Penitenci...|            UN| 979|9: SERVIÇOS DA CO...|97: OUTROS SERVIÇ...|979: OUTROS SERVI...|                 NaN|      NaN|
| 18481|Consultoria e Ass...|            UN|8319|8: SERVIÇOS DE AR...|83: OUTROS SERVIÇ...|831: SERVIÇOS DE ...|8319: OUTROS SERV...|      NaN|
| 19321|Curso / Treinamen...|            UN| 929|9: SERVIÇOS DA CO...|92: SERVIÇOS DA I...|929: OUTROS SERVI...|                 NaN|   

In [26]:
df.columns

['Código',
 'Descrição',
 'Unidade medida',
 'CPC',
 'Seção',
 'Divisão',
 'Grupo',
 'Classe',
 'Subclasse']

In [27]:
df = df.withColumnsRenamed(
    {
     'Código': 'code',
     'Descrição': 'description', 
     'Unidade medida': 'unit_measure', 
     'Seção': 'section', 
     'Divisão': 'division',
     'Grupo': 'group', 
     'Classe': 'class', 
     'Subclasse': 'subclass'
     }
)

In [28]:
df.printSchema()

root
 |-- code: long (nullable = true)
 |-- description: string (nullable = true)
 |-- unit_measure: string (nullable = true)
 |-- CPC: long (nullable = true)
 |-- section: string (nullable = true)
 |-- division: string (nullable = true)
 |-- group: string (nullable = true)
 |-- class: string (nullable = true)
 |-- subclass: double (nullable = true)



# Exporting the data

In [29]:
df.write.mode("overwrite").option("header","True").csv("./data")

In [30]:
spark.read.csv("./data").head()

Row(_c0='code', _c1='description', _c2='unit_measure', _c3='CPC', _c4='section', _c5='division', _c6='group', _c7='class', _c8='subclass')